<a href="https://colab.research.google.com/github/Prathambiradr12345/Insurance-Premium-Category-Predictor/blob/main/fastapi_ml_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, classification_report
import numpy as np



In [ ]:
df=pd.read_csv('insurance.csv')

In [ ]:
df

,age,weight,height,income_lpa,smoker,city,occupation,insurance_premium_category
0,67,119.8,1.56,2.92000,False,Jaipur,retired,High
1,36,101.1,1.83,34.28000,False,Chennai,freelancer,Low
2,39,56.8,1.64,36.64000,False,Indore,freelancer,Low
3,22,109.4,1.55,3.34000,True,Mumbai,student,Medium
4,69,62.2,1.60,3.94000,True,Indore,retired,High
...,...,...,...,...,...,...,...,...
95,36,52.8,1.57,19.64000,False,Indore,business_owner,Low
96,26,113.8,1.54,34.01000,False,Delhi,private_job,Low
97,52,60.8,1.80,44.86000,False,Hyderabad,freelancer,Low
98,27,101.1,1.82,28.30000,False,Kolkata,business_owner,Low


In [ ]:
df.sample(5)

,age,weight,height,income_lpa,smoker,city,occupation,insurance_premium_category
65,46,106.3,1.68,38.07,True,Jaipur,unemployed,High
16,66,70.2,1.59,0.61,False,Pune,retired,Medium
21,69,92.7,1.84,2.91,False,Jalandhar,retired,High
43,72,85.7,1.71,1.56,False,Chennai,retired,Medium
0,67,119.8,1.56,2.92,False,Jaipur,retired,High


In [ ]:
df_feat=df.copy()

In [ ]:
#Feature Engineering=1
df_feat['bmi']=df_feat['weight']/(df_feat['height']**2)

In [ ]:
#Feature Engineering=2
def age_group(age):
    if age<25:
        return 'young'
    elif age<45:
      return 'adult'
    elif age<60:
      return 'middle_aged'
    return 'Senior'

In [ ]:
df_feat['age_group']=df_feat['age'].apply(age_group)

In [ ]:
df_feat

,age,weight,height,income_lpa,smoker,city,occupation,insurance_premium_category,bmi,age_group
0,67,119.8,1.56,2.92000,False,Jaipur,retired,High,49.227482,Senior
1,36,101.1,1.83,34.28000,False,Chennai,freelancer,Low,30.189017,adult
2,39,56.8,1.64,36.64000,False,Indore,freelancer,Low,21.118382,adult
3,22,109.4,1.55,3.34000,True,Mumbai,student,Medium,45.535900,young
4,69,62.2,1.60,3.94000,True,Indore,retired,High,24.296875,Senior
...,...,...,...,...,...,...,...,...,...,...
95,36,52.8,1.57,19.64000,False,Indore,business_owner,Low,21.420747,adult
96,26,113.8,1.54,34.01000,False,Delhi,private_job,Low,47.984483,adult
97,52,60.8,1.80,44.86000,False,Hyderabad,freelancer,Low,18.765432,middle_aged
98,27,101.1,1.82,28.30000,False,Kolkata,business_owner,Low,30.521676,adult


In [ ]:
#Feature Engineering=3
# Feature 3: Lifestyle Risk
def lifestyle_risk(row):
    if row["smoker"] and row["bmi"] > 30:
        return "high"
    elif row["smoker"] or row["bmi"] > 27:
        return "medium"
    else:
        return "low"

In [ ]:
df_feat["lifestyle_risk"] = df_feat.apply(lifestyle_risk, axis=1)

In [ ]:
tier_1_cities = ["Mumbai", "Delhi", "Bangalore", "Chennai", "Kolkata", "Hyderabad", "Pune"]
tier_2_cities = [
    "Jaipur", "Chandigarh", "Indore", "Lucknow", "Patna", "Ranchi", "Visakhapatnam", "Coimbatore",
    "Bhopal", "Nagpur", "Vadodara", "Surat", "Rajkot", "Jodhpur", "Raipur", "Amritsar", "Varanasi",
    "Agra", "Dehradun", "Mysore", "Jabalpur", "Guwahati", "Thiruvananthapuram", "Ludhiana", "Nashik",
    "Allahabad", "Udaipur", "Aurangabad", "Hubli", "Belgaum", "Salem", "Vijayawada", "Tiruchirappalli",
    "Bhavnagar", "Gwalior", "Dhanbad", "Bareilly", "Aligarh", "Gaya", "Kozhikode", "Warangal",
    "Kolhapur", "Bilaspur", "Jalandhar", "Noida", "Guntur", "Asansol", "Siliguri"
]



In [ ]:
##Feature Engineering=4
def city_tier(city):
  if city in tier_1_cities:
    return 1
  elif city in tier_2_cities:
    return 2
  else:
    return 3

In [ ]:
df_feat["city_tier"] = df_feat["city"].apply(city_tier)

In [ ]:
df_feat.drop(columns=['age', 'weight', 'height', 'smoker', 'city'])[['income_lpa', 'occupation', 'bmi', 'age_group', 'lifestyle_risk', 'city_tier', 'insurance_premium_category']].sample(5)


,income_lpa,occupation,bmi,age_group,lifestyle_risk,city_tier,insurance_premium_category
53,30.000000,government_job,29.598247,adult,medium,1,Medium
14,13.505166,government_job,32.800735,middle_aged,medium,3,Medium
77,0.610000,retired,37.818734,Senior,high,1,High
49,2.290000,student,42.701490,young,medium,3,Medium
90,21.070000,business_owner,21.093750,middle_aged,low,1,Low


In [ ]:

# Select features and target
X= df_feat[["bmi", "age_group", "lifestyle_risk", "city_tier", "income_lpa", "occupation"]]
y = df_feat["insurance_premium_category"]

In [ ]:
X

,bmi,age_group,lifestyle_risk,city_tier,income_lpa,occupation
0,49.227482,Senior,medium,2,2.92000,retired
1,30.189017,adult,medium,1,34.28000,freelancer
2,21.118382,adult,low,2,36.64000,freelancer
3,45.535900,young,high,1,3.34000,student
4,24.296875,Senior,medium,2,3.94000,retired
...,...,...,...,...,...,...
95,21.420747,adult,low,2,19.64000,business_owner
96,47.984483,adult,medium,1,34.01000,private_job
97,18.765432,middle_aged,low,1,44.86000,freelancer
98,30.521676,adult,medium,1,28.30000,business_owner


In [ ]:
y

,insurance_premium_category
0,High
1,Low
2,Low
3,Medium
4,High
...,...
95,Low
96,Low
97,Low
98,Low


In [ ]:
# Define categorical and numeric features
categorical_features = ["age_group", "lifestyle_risk", "occupation", "city_tier"]
numeric_features = ["bmi", "income_lpa"]

In [ ]:
# Create column transformer for OHE
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(), categorical_features),
        ("num", "passthrough", numeric_features)
    ]
)

In [ ]:
# Create a pipeline with preprocessing and random forest classifier
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(random_state=42))
])


In [ ]:
# Split data and train model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat', OneHotEncoder(),
                                                  ['age_group',
                                                   'lifestyle_risk',
                                                   'occupation', 'city_tier']),
                                                 ('num', 'passthrough',
                                                  ['bmi', 'income_lpa'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [ ]:
#predict and Evaluate
y_pred=pipeline.predict(X_test)
accuracy_score(y_test,y_pred)

0.8

In [ ]:
X_test.sample(5)

,bmi,age_group,lifestyle_risk,city_tier,income_lpa,occupation
36,21.713266,Senior,low,1,0.53,retired
44,30.078125,middle_aged,high,2,50.00,private_job
84,28.801497,Senior,medium,2,0.62,retired
32,31.495845,middle_aged,medium,2,50.00,private_job
82,17.874812,adult,low,1,12.96,unemployed


In [ ]:
import pickle

#save
pickle_model_path='model.pkl'
with open(pickle_model_path, 'wb') as file:
    pickle.dump(pipeline, file)

In [ ]:
pip show scikit-learn


Name: scikit-learn
Version: 1.6.1
Summary: A set of python modules for machine learning and data mining
Home-page: https://scikit-learn.org
Author: 
Author-email: 
License: BSD 3-Clause License

 Copyright (c) 2007-2024 The scikit-learn developers.
 All rights reserved.

 Redistribution and use in source and binary forms, with or without
 modification, are permitted provided that the following conditions are met:

 * Redistributions of source code must retain the above copyright notice, this
   list of conditions and the following disclaimer.

 * Redistributions in binary form must reproduce the above copyright notice,
   this list of conditions and the following disclaimer in the documentation
   and/or other materials provided with the distribution.

 * Neither the name of the copyright holder nor the names of its
   contributors may be used to endorse or promote products derived from
   this software without specific prior written permission.

 THIS SOFTWARE IS PROVIDED BY THE COPYR